In [69]:
import pandas as pd
import numpy as np
import time

hols = ['2020-03-08',
'2020-03-25',
'2020-03-30',
'2020-03-31']

In [26]:
orders = pd.read_csv("delivery_orders_march.csv")

In [25]:
def rename(s):
    l = ["Metro Manila", "Luzon", "Visayas", "Mindanao"]
    for name in l:
        if name.lower() in s.lower():
            return name
        

In [27]:
orders['buyeraddress'] = orders['buyeraddress'].map(rename) 
orders['selleraddress'] = orders['selleraddress'].map(rename)

In [54]:
orders['pick'] = orders['pick'].map(lambda a:time.strftime('%Y-%m-%d', time.localtime(a)))
orders['1st_deliver_attempt'] = orders['1st_deliver_attempt'].map(lambda a:time.strftime('%Y-%m-%d', time.localtime(a)))
orders['2nd_deliver_attempt'] = orders['2nd_deliver_attempt'].map(lambda a:time.strftime('%Y-%m-%d', time.localtime(a)) if pd.notnull(a) else 0)

In [61]:
sla1_conditions = [
    orders['buyeraddress'].isin(["Visayas", "Mindanao"])| orders['selleraddress'].isin(["Visayas", "Mindanao"]),
    (orders['buyeraddress']=="Metro Manila") & (orders['selleraddress']=="Metro Manila")
]
sla1_choices = [7,3]
orders['sla1'] = np.select(sla1_conditions, sla1_choices, default=5)

In [72]:
orders['attempt1']=np.busday_count(orders['pick'],orders['1st_deliver_attempt'],holidays = hols, weekmask='1111110')

In [74]:
orders['attempt2']=np.busday_count(orders['1st_deliver_attempt'],orders['2nd_deliver_attempt'],holidays = hols, weekmask='1111110')

In [91]:
orders['is_late'] = (orders['sla1'] - orders['attempt1'] < 0) | (orders['attempt2']>3)

In [92]:
ans = orders[['orderid','is_late']]

In [94]:
ans.to_csv('ans.csv',index = False)

In [93]:
ans.is_late.value_counts()

False    2413786
True      762527
Name: is_late, dtype: int64